# Q Learning

In [1]:
import gym
import numpy as np

## Load our game

In [2]:
env = gym.make('FrozenLake-v0')

In [3]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [4]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
# Set learning parameters
lr = .8 #learning rate
y = .95 #gamma
num_episodes = 10000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a) #left Down right up
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a]) # (1-lr)Q[s,a] + lr(r+y*np.max(Q[s1,:]))
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)
    #print(Q) #can you see the procedure of learning Q?
    if (i != 0)&(i%100 == 0):
        print("%d-th Score over time: "%i +  str(sum(rList[i-100:i])/100))

100-th Score over time: 0.0
200-th Score over time: 0.0
300-th Score over time: 0.04
400-th Score over time: 0.0
500-th Score over time: 0.13
600-th Score over time: 0.49
700-th Score over time: 0.39
800-th Score over time: 0.39
900-th Score over time: 0.42
1000-th Score over time: 0.38
1100-th Score over time: 0.46
1200-th Score over time: 0.37
1300-th Score over time: 0.41
1400-th Score over time: 0.39
1500-th Score over time: 0.36
1600-th Score over time: 0.48
1700-th Score over time: 0.41
1800-th Score over time: 0.46
1900-th Score over time: 0.36
2000-th Score over time: 0.4
2100-th Score over time: 0.43
2200-th Score over time: 0.46
2300-th Score over time: 0.43
2400-th Score over time: 0.37
2500-th Score over time: 0.33
2600-th Score over time: 0.37
2700-th Score over time: 0.34
2800-th Score over time: 0.37
2900-th Score over time: 0.38
3000-th Score over time: 0.5
3100-th Score over time: 0.41
3200-th Score over time: 0.45
3300-th Score over time: 0.44
3400-th Score over time:

In [6]:
print("Score over time: " +  str(sum(rList)/num_episodes))

Score over time: 0.3955


In [8]:
print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[2.22060341e-03 2.11174752e-03 2.61828239e-02 2.33961087e-03]
 [6.24791927e-04 9.56195598e-04 1.96770142e-04 8.74026430e-03]
 [4.49906655e-04 4.55861462e-04 6.40634550e-04 2.61848760e-03]
 [6.76243488e-05 3.32217027e-05 1.24281826e-04 2.33264017e-03]
 [9.57568451e-02 5.48614459e-04 1.14366697e-03 4.18397962e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.17977057e-05 1.52086506e-06 3.71590053e-03 9.21536861e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.17155249e-04 3.85500997e-06 5.87860627e-04 2.76028704e-01]
 [2.61123035e-04 2.28089205e-01 1.67722703e-05 6.52498742e-04]
 [5.34412967e-01 1.16248333e-04 0.00000000e+00 5.66672654e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.19448100e-04 2.41125438e-04 2.24913788e-01 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.11350077e-01]
 [0.00000000e+00 0.00000000e+00 0.

# Q learning with a neural network

In [9]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
env = gym.make('FrozenLake-v0')

In [11]:
tf.reset_default_graph()

In [12]:
#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [13]:
init = tf.initialize_all_variables()

# Set learning parameters
y = .99
e = 0.1
num_episodes = 10000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Network
        while j < 99:
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(16)[s:s+1]})
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a[0])
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(16)[s1:s1+1]})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
            rAll += r
            s = s1
            if d == True:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
        if (i != 0)&(i%100 == 0):
            print("%d-th Score over time: "%i +  str(sum(rList[i-100:i])/100))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
100-th Score over time: 0.01
200-th Score over time: 0.03
300-th Score over time: 0.03
400-th Score over time: 0.05
500-th Score over time: 0.12
600-th Score over time: 0.33
700-th Score over time: 0.33
800-th Score over time: 0.52
900-th Score over time: 0.54
1000-th Score over time: 0.46
1100-th Score over time: 0.46
1200-th Score over time: 0.62
1300-th Score over time: 0.66
1400-th Score over time: 0.57
1500-th Score over time: 0.53
1600-th Score over time: 0.65
1700-th Score over time: 0.57
1800-th Score over time: 0.32
1900-th Score over time: 0.65
2000-th Score over time: 0.63
2100-th Score over time: 0.62
2200-th Score over time: 0.64
2300-th Score over time: 0.57
2400-th Score over time: 0.69
2500-th Score over time: 0.74
2600-th Score over time: 0.53
2700-th Score over time: 0.72
2800-th Score over time: 0.57
2900-th Score over time: 0.75
3000-th Score over time: 0.63
3100-th Score over time: 0.65
3200-